In [1]:
import networkx as nx
import sys
import cs6216_taylor_series_identification as TSI
import cs6216_knowledge_graph_creation_and_expansion_experiments as KGCE
import math
import numpy as np
import pandas as pd

In [2]:
# hyperparameters:

series_length = 32

In [3]:
sine_taylor_series = [-1/math.factorial(i)*(i%2 != 0)*((i+1)%4==0)+1/math.factorial(i)*((i+3)%4 == 0) for i in range(series_length)]
cosine_taylor_series = [1/math.factorial(i)*(i%4 == 0) + -1/math.factorial(i)*((i+2)%4 == 0) for i in range(series_length)]
quad_taylor_series = [0+1*(i==2) for i in range(series_length)]
cubic_taylor_series = [0+1*(i==3) for i in range(series_length)]
expon_taylor_series = [1/math.factorial(i)*(i!=0) for i in range(series_length)]

test_dataframe = TSI.knowledge_dataframe_creation([["sine", {'taylor_series': sine_taylor_series}]], 
                             number_of_samples = 500, series_length = series_length)

print(test_dataframe)

print(np.sin(4.83))

                   input label                output ts_0 ts_1 ts_10  \
0     1.0905247349277474  sine    0.8868694745990074  0.0  1.0   0.0   
1       3.82514448926146  sine   -0.6315508624547466  0.0  1.0   0.0   
2      0.470606977779946  sine    0.4534273640594393  0.0  1.0   0.0   
3      4.427868991549799  sine   -0.9597965008098308  0.0  1.0   0.0   
4     1.2106386524844626  sine    0.9358412674475961  0.0  1.0   0.0   
5     2.1261475065211863  sine    0.8497153360617793  0.0  1.0   0.0   
6    0.08618900367591609  sine   0.08608233349771807  0.0  1.0   0.0   
7     3.9981671603900875  sine    -0.755603200896247  0.0  1.0   0.0   
8      2.581128493613879  sine    0.5315794025981164  0.0  1.0   0.0   
9     1.4464788268198359  sine    0.9922825266454227  0.0  1.0   0.0   
10     1.411616455732629  sine    0.9873576128223507  0.0  1.0   0.0   
11    1.3923619059680696  sine     0.984122771926196  0.0  1.0   0.0   
12     4.685170479169416  sine   -0.9996295994641464  0.0  1.0  

/home/ziyu/.pyenv/versions/3.7.7/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
def multiplication(arr1, arr2):
  matrix = np.fliplr(np.outer(np.asarray(arr1, dtype = np.float32), np.asarray(arr2, dtype = np.float32)))
  output = []
  starting_offset = int(np.floor(0.25*(len(arr1)+len(arr2))))
  for i in range(len(arr1)-1, -len(arr1)+1,-1):
    output.append(np.trace(matrix, offset=i))
  return output

def function_compositionality(arr1, arr2): #where both arrays represent functions and function compositionality is done as: arr1(arr2)
  power_results = []
  power_results.append(arr2)
  for i in range(len(arr1)-2):
    power_results.append(multiplication(power_results[i], arr2)[0:len(arr1)])
  power_results = np.asarray(power_results, dtype = np.float32)
  output = np.sum(np.multiply(np.asarray(arr1[1:len(arr1)], dtype=np.float32)[:, np.newaxis],power_results), axis = 1)
  output = np.insert(output, 0, arr1[0])
  print(output)
  return output

In [29]:
def create_initial_knowledge_graph(series_length = series_length):
  #this function creates a simple directed graph with nodes representing inputs and outputs of functions (like X, sin(X), ln(X), sin(X)+ln(X), etc)
  #and edges representing operators 
  G=nx.Graph()
  #start of the graph:
  G.add_node("X", taylor_series = [0+1*(i==1) for i in range(series_length)])
  # 1 step away:]
  G.add_node("sine(X)", taylor_series = [-1/math.factorial(i)*(i%2 != 0)*((i+1)%4==0)+1/math.factorial(i)*((i+3)%4 == 0) for i in range(series_length)])
  G.add_edge("X", "sine(X)", travel = np.subtract([-1/math.factorial(i)*(i%2 != 0)*((i+1)%4==0)+1/math.factorial(i)*((i+3)%4 == 0) 
                                       for i in range(series_length)], [0+1*(i==1) for i in range(series_length)]))
  G.add_node("cosine(X)", taylor_series = [1/math.factorial(i)*(i%4 == 0) + -1/math.factorial(i)*((i+2)%4 == 0) for i in range(series_length)])
  G.add_edge("X", "cosine(X)", travel = np.subtract([1/math.factorial(i)*(i%4 == 0) + -1/math.factorial(i)*((i+2)%4 == 0) 
                                         for i in range(series_length)], [0+1*(i==1) for i in range(series_length)]))
  G.add_node("exponential(X)", taylor_series = [1/math.factorial(i)*(i!=0) for i in range(series_length)])
  G.add_edge("X", "exponential(X)", travel = np.subtract([1/math.factorial(i)*(i!=0) 
                                              for i in range(series_length)], [0+1*(i==1) for i in range(series_length)]))
  G.add_node("quadr(X)", taylor_series =  [0+1*(i==2) for i in range(series_length)])
  G.add_edge("X", "quadr(X)", travel = np.subtract([0+1*(i==2) 
                                        for i in range(series_length)], [0+1*(i==1) for i in range(series_length)]))
  G.add_node("cubic(X)", taylor_series =  [0+1*(i==3) for i in range(series_length)])
  G.add_edge("X", "cubic(X)", travel = np.subtract([0+1*(i==3) 
                                        for i in range(series_length)], [0+1*(i==1) for i in range(series_length)]))
  return G

def print_outcomes(pct):
  possible_answers = set(pct)
  output = []
  for i in possible_answers:
    output.append([i, sum(list(map(int, pct == i)))/len(pct)])
  return output

In [45]:
#expanding the graph through mutiplication of nodes by constant

G = create_initial_knowledge_graph(series_length)

# print([[x,y,z] for x,y,z in G.edges(data = True)])

def graph_expansion_multiplication_constant(G, constant):
    multiplier = [0+constant*(i==0) for i in range(series_length)]
    F = G
    all_nodes = [[x,y] for x,y in F.nodes(data = True)]
    for i in all_nodes:
        new_node_name = str(constant)+"*"+i[0]
        if new_node_name in G:
            continue
        else: 
            F.add_node(new_node_name, taylor_series = multiplication(multiplier, i[1]["taylor_series"]))
            F.add_edge(i[0], new_node_name, travel = np.subtract(multiplication(multiplier, i[1]["taylor_series"])[0:32],
                                                                 i[1]["taylor_series"]))
    return F

F = graph_expansion_multiplication_constant(G, 3)
print([[x,y] for x,y in F.nodes(data = True)])
print([[x,y,z] for x,y,z in F.edges(data = True)])

[['X', 'sine(X)', {'travel': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.66666667e-01,
        0.00000000e+00,  8.33333333e-03,  0.00000000e+00, -1.98412698e-04,
        0.00000000e+00,  2.75573192e-06,  0.00000000e+00, -2.50521084e-08,
        0.00000000e+00,  1.60590438e-10,  0.00000000e+00, -7.64716373e-13,
        0.00000000e+00,  2.81145725e-15,  0.00000000e+00, -8.22063525e-18,
        0.00000000e+00,  1.95729411e-20,  0.00000000e+00, -3.86817017e-23,
        0.00000000e+00,  6.44695028e-26,  0.00000000e+00, -9.18368986e-29,
        0.00000000e+00,  1.13099629e-31,  0.00000000e+00, -1.21612504e-34])}], ['X', 'cosine(X)', {'travel': array([ 1.00000000e+00, -1.00000000e+00, -5.00000000e-01,  0.00000000e+00,
        4.16666667e-02,  0.00000000e+00, -1.38888889e-03,  0.00000000e+00,
        2.48015873e-05,  0.00000000e+00, -2.75573192e-07,  0.00000000e+00,
        2.08767570e-09,  0.00000000e+00, -1.14707456e-11,  0.00000000e+00,
        4.77947733e-14,  0.00000000e+

In [47]:
#expanding the graph through mutiplication of nodes by constant

G = create_initial_knowledge_graph(series_length)

def graph_expansion_multiplication_nodes(G):
    F = G
    all_nodes = [[x,y] for x,y in F.nodes(data = True)]
    for i in range(len(all_nodes)):
        for j in range(i, len(all_nodes)):
            new_name = all_nodes[i][0] + "*" + all_nodes[j][0]
            new_taylor_series = multiplication(all_nodes[i][1]["taylor_series"], all_nodes[j][1]["taylor_series"])[0:32]
            F.add_node(new_name, taylor_series = new_taylor_series)
            F.add_edge(all_nodes[i][0], new_name, travel = np.subtract(new_taylor_series, all_nodes[i][1]["taylor_series"]))
            F.add_edge(all_nodes[j][0], new_name, travel = np.subtract(new_taylor_series, all_nodes[j][1]["taylor_series"]))
    return F

F = graph_expansion_multiplication_nodes(G)
print([[x,y] for x,y in F.nodes(data = True)])

[['X', {'taylor_series': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}], ['sine(X)', {'taylor_series': [0.0, 1.0, 0.0, -0.16666666666666666, 0.0, 0.008333333333333333, 0.0, -0.0001984126984126984, 0.0, 2.7557319223985893e-06, 0.0, -2.505210838544172e-08, 0.0, 1.6059043836821613e-10, 0.0, -7.647163731819816e-13, 0.0, 2.8114572543455206e-15, 0.0, -8.22063524662433e-18, 0.0, 1.9572941063391263e-20, 0.0, -3.868170170630684e-23, 0.0, 6.446950284384474e-26, 0.0, -9.183689863795546e-29, 0.0, 1.1309962886447716e-31, 0.0, -1.216125041553518e-34]}], ['cosine(X)', {'taylor_series': [1.0, 0.0, -0.5, 0.0, 0.041666666666666664, 0.0, -0.001388888888888889, 0.0, 2.48015873015873e-05, 0.0, -2.755731922398589e-07, 0.0, 2.08767569878681e-09, 0.0, -1.1470745597729725e-11, 0.0, 4.779477332387385e-14, 0.0, -1.5619206968586225e-16, 0.0, 4.110317623312165e-19, 0.0, -8.896791392450574e-22, 0.0, 1.6117375710961184e-24, 0.0, -2.4795962632247976e-27, 0.0, 3.2798

In [ ]:
#typical usage: 
G = create_initial_knowledge_graph(series_length)
prediction = TSI.make_prediction(TSI.knowledge_dataframe_creation([[x,y] for x,y in G.nodes(data = True)], 500,series_length), test_dataframe[["input", "output"]].values, series_length)
#note that more data is needed to run with longer taylor series
percentage = TSI.match_prediction_to_series(prediction, [[x,y] for x,y in G.nodes(data = True)])
print(print_outcomes(percentage))